In [1]:
import pandas as pd
import numpy as np
from io import BytesIO
import requests
from urllib.parse import urlencode
import urllib
from requests.auth import HTTPBasicAuth
from requests.exceptions import ChunkedEncodingError
import os
import json
import yadisk
from datetime import datetime, date, timedelta
import locale
from time import sleep
import shutil
import gc
import turbodbc
from turbodbc import connect
import gc
from pandas.api.types import is_string_dtype
import numpy as np
from sqlalchemy import create_engine
import pyodbc
import warnings


pd.options.mode.chained_assignment = None


# параметры подключения к нашей БД, в которую сохранить итоговые таблицы
server_local = 'REGRU-54856\MSSQLSERVER01'
database_local = 'custom_tables_NL_2208' #'custom_tables_NL_2208' - БД для тестовой загрузки / 'NL_db_27082024' - БД для дашбодра НЛ
conn_str_local = 'DRIVER={SQL Server};SERVER='+server_local+';DATABASE='+database_local+';'

# указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\32_NL_Yandex_Disk'
file_name = 'yandex_token.txt'

# сохраняем токен в отдельную переменную
yandex_token = open(os.path.join(file_path, file_name), encoding='utf-8').read()
# формируем заголовки для дальнейших запросов
headers = {
    'Content-Type': 'application/json', 
    'Accept': 'application/json', 
    'Authorization': f'OAuth {yandex_token}'
}

# указываем путь к основной папке, в которой храняться папки с флайтами
main_folder = '/NL_отчеты_для_дашборда'

# УРЛ для операций над опубликованными папками и файлами на Яндекс Диске
base_public_url = 'https://cloud-api.yandex.net/v1/disk/public/resources?'  
public_key = 'https://disk.yandex.ru/d/2iYr8zMm9Qgeug' # обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета

download_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' # УРЛ для скачивания эксель файлов из Яндекс Диска
delete_url = 'https://cloud-api.yandex.net/v1/disk/resources?' # УРЛ для удаления папок и файлов на Яндекс Диске

final_url = base_public_url + urlencode(dict(public_key=public_key))  # Формируем УРЛ для получения списка папок и файлов
res = requests.get(final_url)  # Отправлем запрос на Яндекс диск, чтобы получить название папок и public key для скачивания файлов

print(res.status_code)

yandex_folders = res.json() # парсим ответ

200


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)

# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_rows', None)

pd.options.mode.chained_assignment = None

In [3]:
# Создаем функцию для получения содержимого Яндекс диска
#  на входе принимаем 
# - адрес запроса для скачивания файла
# - public key, который мы получаем из ответа Яндекс при запросе информации по конкретной папке
# - адрес папки
# - лимит файлов (максимальное кол-во 80)
def get_yandex_disk_responce(request_url, public_key, folder_path, limit=80):
    res_url = request_url + urlencode(dict(public_key=public_key, path=folder_path, limit=limit)) # формируем строку запроса
    res = requests.get(res_url) # отправляем запрос на сервер
    yandex_responce = res.json() # получаем ответ и преобразуем его в json

    return yandex_responce

In [4]:
public_key = yandex_folders['public_key']  # из ответа Яндекс забираем public_key, чтобы использовать его для скачивания файлов

for i in range(len(yandex_folders['_embedded']['items'])): # через цикл проходим по ответу Яндекса и забираем названия вложенных папок
    file_type = yandex_folders['_embedded']['items'][i]['type']
    if file_type=='dir':   # если находим файлы с типом dir (папка), то забираем путь к этой папке
        folder_path = yandex_folders['_embedded']['items'][i]['path']
        print(folder_path)
        yandex_responce = get_yandex_disk_responce(base_public_url, public_key, folder_path) # отправляем запрос, чтобы получить содержимое папки

        # Через цикл проходим по папке с файлами
        # Нас интересуют файлы эксель. Причем каждая экселька будет парситься по своему, т.к. они относятся к разным рекламным площадкам
        
        # Проходим через цикл по содержимому папки (отдельный флайт)
        for i in range(len(yandex_responce['_embedded']['items'])):
            file_info = yandex_responce['_embedded']['items'][i]
            if file_info['type']=='file':  # если документ является фалйом(не папкой или изображением), то забираем его название 
                file_name = file_info['name'] # сохраняем название файла
                if 'xls' in file_name: # еслит тип файла является xlsx, то уберем расширение и будем его использовать в качесвте названия отчета
                    file_path = file_info['path']
                    
                    report_name = '.'.join(file_name.split('.')[:-1]) # убираем .xlsx из названия файла
                    print(report_name)
                    res_file_link = get_yandex_disk_responce(download_url, public_key, file_path) # получаем ссылку на скачивание отчета
                    download_response = requests.get(res_file_link['href'])

/01_new_sources_files
one_target_video
/02_regular_reports


In [5]:
data_link = download_response.content

In [6]:
sheet_names = pd.ExcelFile(BytesIO(data_link))

In [7]:
sheet_names.sheet_names

['Отчет по всем РК', 'Отчет по дням', 'Worksheet 1']

In [8]:
report_type = 'test'

In [14]:
cols_range = 'A:G' # задаем диапазон полей, которые нам нужны
df = pd.read_excel(BytesIO(data_link), sheet_name='Отчет по всем РК', header=None, usecols=cols_range,
               names=['product', 'impressions', '25', '50', '75', '100', 'VTR']) 
df = df[['product', 'impressions', '25', '50', '75', '100']] # оставляем только нужные поля

# на этом листе нет разбивки по дням
# в самом верху таблицы присутствует строка с указанием периода загрузки
# из этой строки мы заберем период и сохраним его в двух отдельных полях - начало и окончание периода
period = df['product'][0] # забираем текст из ячейки, которая содержит период
period = period.split('период ', 1)[1]
start_period = period.split(' - ', 1)[0] # начало периода отчета
end_period = period.split(' - ', 1)[1] # окончание периода отчета

# теперь нам необходимо забрать статистику досмотров в разбивке по каждому продукту
# для этого получим индекс строк, которые нужно сохранить
df = df.fillna('') # заполняем пустые строки
target_rows_list = list(df[df['product'].str.contains('Досмотры')].index) # создаем список индексов строк, которые содержат данные
df = df.iloc[target_rows_list] # передаем список индексов, чтобы отфильтровать датаФрейм

# теперь необходимо извлечь из ячейки название продукта
df['product'] = df['product'].apply(lambda x: x.split('Досмотры ', 1)[1])
# добавляем название источника, даты начала и окончания
df['source'] = 'one target'
df['start_period'] = start_period
df['end_period'] = end_period
df['product'] = df['product'].str.lower().str.strip()
df['report_type'] = report_type

In [15]:
df

,product,impressions,25,50,75,100,source,start_period,end_period,report_type
14,6s,440585,440581,401255,389087,349872.0,one target,2024-07-17,2024-07-31,test
18,6snew,142215,142213,138764,129097,122412.0,one target,2024-07-17,2024-07-31,test
25,15s,438086,438002,399471,301982,289762.0,one target,2024-07-17,2024-07-31,test
29,15snew,149442,149440,139766,120994,115642.0,one target,2024-07-17,2024-07-31,test


In [ ]:
# Отдельно создаем функцию для парсинга досмотров по источнику One target
# на входе функция принимает только ссылку для скачивания таблицы
def get_one_target_views(data_link, report_type):
    cols_range = 'A:G' # задаем диапазон полей, которые нам нужны
    df = pd.read_excel(BytesIO(data_link), sheet_name='Отчет по всем РК', header=None, usecols=cols_range,
                   names=['product', 'impressions', '25', '50', '75', '100', 'VTR']) 
    df = df[['product', 'impressions', '25', '50', '75', '100']] # оставляем только нужные поля

    # на этом листе нет разбивки по дням
    # в самом верху таблицы присутствует строка с указанием периода загрузки
    # из этой строки мы заберем период и сохраним его в двух отдельных полях - начало и окончание периода
    period = df['product'][0] # забираем текст из ячейки, которая содержит период
    period = period.split('период ', 1)[1]
    start_period = period.split(' - ', 1)[0] # начало периода отчета
    end_period = period.split(' - ', 1)[1] # окончание периода отчета

    # теперь нам необходимо забрать статистику досмотров в разбивке по каждому продукту
    # для этого получим индекс строк, которые нужно сохранить
    df = df.fillna('') # заполняем пустые строки
    target_rows_list = list(df[df['product'].str.contains('Досмотры')].index) # создаем список индексов строк, которые содержат данные
    df = df.iloc[target_rows_list] # передаем список индексов, чтобы отфильтровать датаФрейм

    # теперь необходимо извлечь из ячейки название продукта
    df['product'] = df['product'].apply(lambda x: x.split('Досмотры ', 1)[1])
    # добавляем название источника, даты начала и окончания
    df['source'] = 'one target'
    df['start_period'] = start_period
    df['end_period'] = end_period
    df['product'] = df['product'].str.lower().str.strip()
    df['report_type'] = report_type
    return df